In [12]:
import pandas as pd
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
# load dataset
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
test_or_data = data_test
data.head()
# numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# numerical_vars = list(data.select_dtypes(include=numerics).columns)
# data = data[numerical_vars]
data.shape

# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['TARGET'], axis=1),
    data['TARGET'],
    test_size=0.3,
    stratify = data['TARGET'],
    random_state = 888)
X_train.shape, y_train.shape

# Constant Removal (X_train)
from sklearn.feature_selection import VarianceThreshold
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_train)

constant_list = [not temp_feat for temp_feat in constant_filter.get_support()]
X_train_filter = constant_filter.transform(X_train)
X_test_filter = constant_filter.transform(X_test)
# X_train_filter.shape,X_train.shape
data_test = constant_filter.transform(data_test)

In [13]:
data_test.shape, X_train_filter.shape, X_test_filter.shape

((75818, 330), (53214, 330), (22806, 330))

In [14]:

# Quasi Constant Removal (X_train)
quasi_constant_filter = VarianceThreshold(threshold=0.01)
quasi_constant_filter.fit(X_train)

constant_list = [not temp_feat for temp_feat in quasi_constant_filter.get_support()]

X_train_quasi_filter  = quasi_constant_filter.transform(X_train)
X_test_quasi_filter  = quasi_constant_filter.transform(X_test)
data_test = quasi_constant_filter.transform(test_or_data)

X_train_quasi_filter.shape
data_test.shape


(75818, 269)

In [15]:

# Duplicate Feature Removal (X_train)
X_train_T = X_train_quasi_filter.T
type(X_train_T)
X_train_T = pd.DataFrame(X_train_T)
X_train_T.shape
X_train_T.duplicated().sum()
duplicated_features = X_train_T.duplicated()
features_to_keep = [not index for index in duplicated_features]
X_train_unique = X_train_T[features_to_keep].T
X_train_unique.shape, X_train.shape


for i in range(len(duplicated_features) - 1, 0, -1):
    if duplicated_features[i] == True:
        data_test = np.delete(data_test, i, axis=1)
        X_test_quasi_filter = np.delete(X_test_quasi_filter, i, axis=1)

X_test_quasi_filter.shape, data_test.shape

((22806, 251), (75818, 251))

In [16]:




X_test = X_test_quasi_filter
X_train = X_train_unique


# from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
# from sklearn.ensemble import RandomForestClassifier
# best_model = RandomForestClassifier(max_depth=23, min_samples_split=150, n_estimators=100)
# best_model.fit(X_train,y_train)
# print(f'Test ROC AUC Score: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1])}')
# print(f'Train ROC AUC  Score: {roc_auc_score(y_train, best_model.predict_proba(X_train)[:,1])}')

from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
import xgboost as xgb
best_model = xgb.XGBClassifier(learning_rate=0.11,max_depth=4,min_child_weight=4)
best_model.fit(X_train,y_train)
# print(f'Test ROC AUC Score: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1])}')
# print(f'Train ROC AUC Score: {roc_auc_score(y_train, best_model.predict_proba(X_train)[:,1])}')
y_predict = best_model.predict_proba(data_test)
np.savetxt("result3.csv", y_predict[:,1], delimiter=',')

D:\Program2\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:21:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
